In [1]:
range(M, N, [M|Ns]) :- M < N, M1 is M + 1, range(M1, N, Ns).
range(N, N, [N]).
?- range(1, 3, Ns).

% Asserting clauses for user:range/3


Ns = [1,2,3]

In [2]:
queens(N, Qs) :- generate(N, Qs), safe(Qs).
generate(N, Qs) :- range(1, N, Ns), permutation(Ns, Qs).
safe([Q|Qs]) :- safe(Qs), not(attack(Q, Qs)).
safe([]).
attack(X, Xs) :- attack(X, 1, Xs).
attack(X, N, [Y|Ys]) :- X is Y + N ; X is Y - N.
?- queens(4, Qs).
?- retry.

% Asserting clauses for user:queens/2


% Asserting clauses for user:generate/2


% Asserting clauses for user:safe/1


% Asserting clauses for user:attack/2


% Asserting clauses for user:attack/3


Qs = [2,4,1,3]

% Retrying goal: queens(4,Qs)


Qs = [3,1,4,2]

## １６章　限定節文法による構文解析

### 文脈自由文法
16.1の文脈自由文法を定義する。

終端記号はカギ括弧で括る。
文法がさりげなく、右再帰構文になっていることに注意！


In [3]:
% 文法規則
sentence --> noun_phrase, verb_phrase.
noun_phrase --> determiner, noun_phrase2.
noun_phrase --> noun_phrase2.
noun_phrase2 --> adjective, noun_phrase2.
noun_phrase2 --> noun.
verb_phrase --> verb.
verb_phrase --> verb, noun_phrase.
% 語彙
determiner --> [the].   adjective --> [decorated].
determiner --> [a].
noun --> [pieplate].    verb --> [surprise].
noun --> [surprise].

% Asserting clauses for user:sentence/2


% Asserting clauses for user:noun_phrase/2


% Asserting clauses for user:noun_phrase2/2


% Asserting clauses for user:verb_phrase/2


% Asserting clauses for user:determiner/2


% Asserting clauses for user:adjective/2


% Asserting clauses for user:noun/2


% Asserting clauses for user:verb/2


分脈自由文法は、S\S0の差分リストを引数とする。prolog文に変換される。
```prolog
sentence --> noun_phrase, verb_phrase.
```
これをそのままprolog規則に変換すると
```prolog
sentence(S) :- append(NP, VP, S), noun_pharase(NP), verb_phrase(VP).
```
となるが、appendの処理効率がわるいため、差分リストで表すと以下の様になる。
```prolog
sentence(S\S0) :- noun_phrase(S\S1), verb_phrase(S1\S0).
```
さらにAs\Bsは、headとtailのリストに分解されるので、
```prolog
sentence(S, S0) :- noun_phrase(S, S1), verb_phrase(S1, S0).
```
となる。

listing関数を使って、sentenceの変換結果を見ると、上記の変換が行われていることが確認できる。

In [4]:
?- listing(sentence).


:- dynamic sentence/2.

sentence(A, C) :-
    noun_phrase(A, B),
    verb_phrase(B, C).


true

### 文法規則からProlog節への変換

In [29]:
% translate(GrammerRule, PrologClause) :-
%   PrologClauseは、文脈自由文法の規則GrammerRuleと等価なPrologプログラムである。
translate((Lhs --> Rhs), (Head :- Body)) :- 
    translate(Lhs, Head, [Xs, Ys]), translate(Rhs, Body, [Xs, Ys]).
translate((A, B), (A1, B1), [Xs, Ys]) :- 
    translate(A, A1, [Xs, Xs1]), translate(B, B1, [Xs1, Ys]).
translate(A, A1, S) :-
    non_terminal(A), functor(A1, A, 1), arg(1, A1, S).
translate(Xs, true, S) :-
    terminals(Xs), sequence(Xs, S).
non_terminal(A) :- atom(A).
terminals([X|Xs]).
%sequence([X|Xs], [[X|S], S0]) :- sequence(Xs, [S, S0]).
%sequence([], [Xs, Xs]).

Previously defined clauses of user:translate/2 were retracted:
:- dynamic translate/2.

translate((A-->C),  (B:-D)) :-
    translate(A, B, E, F),
    translate(C, D, E, F).

% Asserting clauses for user:translate/2


Previously defined clauses of user:translate/3 were retracted:
:- dynamic translate/3.

translate(A, B, C) :-
    non_terminal(A),
    functor(B, A, 1),
    arg(1, B, C).
translate(A, true, B) :-
    terminals(A),
    sequence(A, B).

% Asserting clauses for user:translate/3


Previously defined clauses of user:non_terminal/1 were retracted:
:- meta_predicate non_terminal(:).

system:non_terminal(Spec) :-
    '$set_pattr'(Spec, pred, non_terminal(true)).

%   NOTE: system definition has been overruled for non_terminal/1
:- dynamic non_terminal/1.

non_terminal(A) :-
    atom(A).

% Asserting clauses for user:non_terminal/1


Previously defined clauses of user:terminals/1 were retracted:
:- dynamic terminals/1.

terminals([_|_]).

% Asserting clauses for user:terminals/1


In [30]:
?- translate((s --> n, v), (H :- B)), print(H), print(:-), print(B).

s([_15890,_15896]):-n([_15890,_15930]),v([_15930,_15896])

H = s([_15890,_15896]),
B = n([_15890,_15930]),v([_15930,_15896])

In [31]:
s([_24198,_24204]):-n([_24198,_24238]),v([_24238,_24204]).

Previously defined clauses of user:s/1 were retracted:
:- dynamic s/1.

s([A, C]) :-
    n([A, B]),
    v([B, C]).

% Asserting clauses for user:s/1


In [32]:
?- listing(s).

:- dynamic s/1.

s([A, C]) :-
    n([A, B]),
    v([B, C]).


true

## or(；)を使わない実装

In [8]:
queens(N, Qs) :- generate(N, Qs), safe(Qs).
generate(N, Qs) :- range(1, N, Ns), permutation(Ns, Qs).
safe([Q|Qs]) :- safe(Qs), not(attack(Q, Qs)).
safe([]).
attack(X, Xs) :- attack(X, 1, Xs).
attack(X, N, [Y|Ys]) :- X is Y + N.
attack(X, N, [Y|Ys]) :- X is Y - N.
?- queens(4, Qs).
?- retry.

Previously defined clauses of user:queens/2 were retracted:
:- dynamic queens/2.

queens(A, B) :-
    generate(A, B),
    safe(B).

% Asserting clauses for user:queens/2


Previously defined clauses of user:generate/2 were retracted:
:- dynamic generate/2.

generate(A, C) :-
    range(1, A, B),
    permutation(B, C).

% Asserting clauses for user:generate/2


Previously defined clauses of user:safe/1 were retracted:
:- dynamic safe/1.

safe([B|A]) :-
    safe(A),
    not(attack(B, A)).
safe([]).

...

Previously defined clauses of user:attack/3 were retracted:
:- dynamic attack/3.

attack(A, C, [B|_]) :-
    (   A is B+C
    ;   A is B-C
    ).

% Asserting clauses for user:attack/3


Qs = [2,4,1,3]

% Retrying goal: queens(4,Qs)


Qs = [3,1,4,2]

### 差分リストの表現
X\Yが差分リストの表記になっているが、これを[X, Y]として実行しても問題ないかためす。

差分リストのメリットは、リストのコピーが不要で、tail(尾部)は、ポインターとして働くことです。

In [9]:
append_dl([Xs, Ys], [Ys, Zs], [Xs, Zs]).
?- append_dl([[a, b, c|Xs], Xs], [[1, 2], []], Ys).

% Asserting clauses for user:append_dl/3


Xs = [1,2],
Ys = [[a,b,c,1,2],[]]

In [10]:
s --> np,vp.

np --> det,n.

vp --> v,np.
vp --> v.

det --> [the].
det --> [a].

n --> [woman].
n --> [man].

v --> [shoots].

% Asserting clauses for user:s/2


% Asserting clauses for user:np/2


% Asserting clauses for user:vp/2


% Asserting clauses for user:det/2


% Asserting clauses for user:n/2


% Asserting clauses for user:v/2


In [11]:
?- s([a,woman,shoots,a,man],[]).

true

In [12]:
?- s(X,[]).

X = [the,woman,shoots,the,woman]

In [13]:
?- np([a,woman],[]).

true

In [14]:
?- np(X,[]).

X = [the,woman]

In [15]:
?- listing(s).


:- dynamic s/1.

s([A, C]) :-
    n([A, B]),
    v([B, C]).

:- dynamic s/2.

s(A, C) :-
    np(A, B),
    vp(B, C).


true

### 16.5の例題

In [16]:
number(0) --> [zero].
number(N) --> xxx(N).
xxx(N) --> digit(D), [hundred], rest_xxx(N1), { N is D*100 + N1}.
xxx(N) --> xx(N).
rest_xxx(0) --> [].
rest_xxx(N) --> [and], xx(N).

xx(N) --> digit(N).
xx(N) --> teen(N).
xx(N) --> tens(T), rest_xx(N1), {N is T + N1}.

rest_xx(0) --> [].
rest_xx(N) --> digit(N).

digit(1) --> [one].     teen(10) --> [ten].
digit(2) --> [two].     teen(11) --> [eleven].
digit(3) --> [three].   teen(12) --> [twelve].
digit(4) --> [four].    teen(13) --> [thirteen].
digit(5) --> [five].    teen(14) --> [fourteen].
digit(6) --> [six].     teen(15) --> [fifteen].
digit(7) --> [seven].   teen(16) --> [sixteen].
digit(8) --> [eight].   teen(17) --> [seventeen].
digit(9) --> [nine].    teen(18) --> [eighteen].
                        teen(19) --> [nineteen].
tens(20) --> [twenty].
tens(30) --> [thirty].
tens(40) --> [fourty].
tens(50) --> [fifty].
tens(60) --> [sixty].
tens(70) --> [seventy].
tens(80) --> [eighty].
tens(90) --> [ninety].


% Asserting clauses for user:number/3


% Asserting clauses for user:xxx/3


% Asserting clauses for user:rest_xxx/3


% Asserting clauses for user:xx/3


% Asserting clauses for user:rest_xx/3


% Asserting clauses for user:digit/3


% Asserting clauses for user:teen/3


% Asserting clauses for user:tens/3


In [17]:
?- listing(number).

%   Foreign: system:number/1

:- dynamic number/3.

number(0, [zero|A], A).
number(A, B, C) :-
    xxx(A, B, C).


true

In [18]:
?- number(66, Ns, []).

Ns = [sixty,six]

### 順番と選択
dcgでは、終端子、非終端子の並びをカンマで区切って指定し、選択しは同名の非終端子に複数の定義を記述することで実現する。

left recursionしてはいけない！

文法を右結合に変更

In [19]:
expr --> term, addterm.
addterm --> [].
addterm --> [+], expr.
term --> factor, multfactor.
multfactor --> [].
multfactor --> [*], term.
factor --> [I], {integer(I)}.
factor --> ['('], expr, [')'].

% Asserting clauses for user:expr/2


% Asserting clauses for user:addterm/2


% Asserting clauses for user:term/2


% Asserting clauses for user:multfactor/2


% Asserting clauses for user:factor/2


In [20]:
?- expr([4,*,5,+,1], []).

true

In [21]:
?- listing(expr).

:- dynamic expr/2.

expr(A, C) :-
    term(A, B),
    addterm(B, C).


true

### 計算結果を返す

In [22]:
expr(N) --> term(X), addterm(Y), { N is X + Y}.
addterm(0) --> [].
addterm(N) --> [+], expr(N).
term(N) --> factor(X), multfactor(Y), {N is X * Y}.
multfactor(1) --> [].
multfactor(S) --> [*], term(S).
factor(I) --> [I], {integer(I)}.

% Asserting clauses for user:expr/3


% Asserting clauses for user:addterm/3


% Asserting clauses for user:term/3


% Asserting clauses for user:multfactor/3


% Asserting clauses for user:factor/3


In [23]:
?- listing(expr).

:- dynamic expr/2.

expr(A, C) :-
    term(A, B),
    addterm(B, C).

:- dynamic expr/3.

expr(C, A, F) :-
    term(D, A, B),
    addterm(E, B, G),
    C is D+E,
    F=G.


true

In [24]:
?- expr(Y, [1,+,2,*,3], []).

Y = 7

### 以前作成したANTLRの文法を実装
選択子は、（|）を使い、空を[]で指定する。

In [25]:
expression(N) -->
    product(X), 
    ( [],
        {N is X}
    | [+], expression(Y), 
        {N is X + Y} 
    | [-], expression(Y), 
        {N is X - Y}
    ).
product(N) -->
    power(X), 
    ( [],
        { N is X}
    | [*], product(Y),
        { N is X * Y}
    | [/], product(Y),
        { N is X / Y}
    ).
power(N) -->
    factor(X),
    ([],
        { N is X}
    | ['^'], power(Y),
        { N is X**Y}
    ).
factor(I) --> [I], {number(I)}.
    

% Asserting clauses for user:expression/3


% Asserting clauses for user:product/3


% Asserting clauses for user:power/3


Previously defined clauses of user:factor/3 were retracted:
:- dynamic factor/3.

factor(A, [A|C], B) :-
    integer(A),
    B=C.

% Asserting clauses for user:factor/3


In [26]:
?- expression(N, [1, +, 2, '^', 3, *, 4], []).

N = 33

In [27]:
?- listing(expression).

:- dynamic expression/3.

expression(C, A, E) :-
    product(D, A, B),
    (   B=F,
        C is D,
        E=F
    ;   B=[+|G],
        expression(H, G, I),
        C is D+H,
        E=I
    ;   B=[-|J],
        expression(H, J, K),
        C is D-H,
        E=K
    ).


true

### 構文木の生成


In [28]:
parse(Source, Structure) :- expression(Structure, Source, []).
expression(N) -->
    product(X),         
    ( [],
        { N = node("product", X) }
    | [+], expression(Y1), 
        { N = node("+", X, Y1) }
    | [-], expression(Y2), 
        { N = node("-", X, Y2) }
    ).
product(N) -->
    power(X),
    ( [],
        { N = node("product", X) }
    | [*], product(Y1),
        { N = node("*", X, Y1) }
    | [/], product(Y2),
        { N = node("/", X, Y2) }
    ).
power(N) -->
    factor(X),
    ([],
        { N = node("power", X) }
    | ['^'], power(Y1),
        { N = node("^", X, Y1)}
    ).
factor(node("factor", I)) --> [I], {number(I)}.

% Asserting clauses for user:parse/2


Previously defined clauses of user:expression/3 were retracted:
:- dynamic expression/3.

expression(C, A, E) :-
    product(D, A, B),
    (   B=F,
        C is D,
        E=F
    ;   B=[+|G],
        expression(H, G, I),
        C is D+H,
        E=I
    ;   B=[-|J],
        expression(H, J, K),
        C is D-H,
        E=K
    ).

% Asserting clauses for user:expression/3


Previously defined clauses of user:product/3 were retracted:
:- dynamic product/3.

product(C, A, E) :-
    power(D, A, B),
    (   B=F,
        C is D,
        E=F
    ;   B=[*|G],
        product(H, G, I),
        C is D*H,
        E=I
    ;   B=[/|J],
        product(H, J, K),
        C is D/H,
        E=K
    ).

% Asserting clauses for user:product/3


Previously defined clauses of user:power/3 were retracted:
:- dynamic power/3.

power(C, A, E) :-
    factor(D, A, B),
    (   B=F,
        C is D,
        E=F
    ;   B=[^|G],
        power(H, G, I),
        C is D**H,
        E=I
    ).

% Asserting clauses for user:power/3


Previously defined clauses of user:factor/3 were retracted:
:- dynamic factor/3.

factor(A, [A|C], B) :-
    number(A),
    B=C.

% Asserting clauses for user:factor/3


In [29]:
?- parse([1, +, 2, '^', 3, *, 4], Tree).

Tree = node(+,node(product,node(power,node(factor,1))),node(product,node(*,node(^,node(factor,2),node(power,node(factor,3))),node(product,node(power,node(factor,4))))))

```text
node(+,                                 => +                                    => 1 + 32   => 33
    node(product,
        node(power,
            node(factor,1)              => 1                                    ↑
        )
    ),
    node(product,
        node(*,                         => *                    => 8*4  => 32   ↑
            node(^,                     => ^    => 2^3  => 8    ↑
                node(factor,2),         => 2    ↑
                    node(power,         
                        node(factor,3)  => 3    ↑
                    )
                ),
                node(product,
                    node(power,
                        node(factor,4)  => 4                    ↑
                    )
                )
            )
        )
    )
```

In [30]:
?- listing(expression).

:- dynamic expression/3.

expression(C, A, E) :-
    product(D, A, B),
    (   B=F,
        C=node("product", D),
        E=F
    ;   B=[+|G],
        expression(H, G, I),
        C=node("+", D, H),
        E=I
    ;   B=[-|J],
        expression(K, J, L),
        C=node("-", D, K),
        E=L
    ).


true

## １７章　２階プログラミング
個体ではなく、集合やその属性を扱う技法は、「２階プログラミング」と呼ばれます。

特に17.3節で扱っている２階述語では「述語名」が変数である様な規則を用いることができます。

In [31]:
male(abraham).  male(haran).    female(yiscah).
male(isaac).    male(nachor).   female(milcah).
male(lot).

father(abraham, issac).
father(haran, lot).
father(haran, milcah).
father(haran, yiscah).

% Asserting clauses for user:male/1


% Asserting clauses for user:female/1


% Asserting clauses for user:father/2


Xsの各要素が属性Pを持つ時に真を返すhas_propertyをそのまま定義すると、以下の様になる。

しかし、この規則はPがfunctor（オペレータ）である必要があり、コンパイルエラーとなります(コメント%を外して実行)。

In [32]:
%has_property([X|Xs], P) :- P(X), has_property(Xs, P).
%has_property([], P).
%?- has_property([abraham, haran], male).

そこで、Xにfunctor（述語）Pを適用するapply(P, X)をcallを使って定義すると、期待通りに動作するようになります。

In [33]:
has_property([X|Xs], P) :- apply(P, X), has_property(Xs, P).
has_property([], P).
apply(P, X) :- call(P, X).

?- has_property([abraham, haran], male).

% Asserting clauses for user:has_property/2


true

述語Pの引数がリストの場合、univを使って以下の様にapply(P, Xs)を定義します。

In [34]:
apply(P, Xs) :- Goal =.. [P|Xs], Goal.

?- has_property([[1, 2], [1, 3]], <).

Previously defined clauses of user:apply/2 were retracted:
:- meta_predicate'$apply':apply(:,+).

'$apply':apply(M:Name, Extra) :-
    atom(Name),
    !,
    compound_name_arguments(G, Name, Extra),
    call(M:G).
'$apply':apply(M:Goal, Extra) :-
    compound(Goal),
    !,
    compound_name_arguments(Goal, Name, Args0),
    '$append'(Args0, Extra, Args),
    compound_name_arguments(G, Name, Args),
    call(M:G).
'$apply':apply(_:Goal, _Extra) :-
    '$type_error'(callable, Goal).

%   NOTE: system definition has been overruled for apply/2
:- dynamic apply/2.

apply(A, B) :-
    call(A, B).

% Asserting clauses for user:apply/2


true

## １９章　メタ・インタプリタ

### 19.1 簡単なメタ・インタプリタ
Pure(純)Prologのメタ・インタプリタをリスト19.1に合わせて実行してみました。

solve(ture).がうまく動作せず、カットオペレータを挿入することで対処しました。またmemberも組み込みを使うと動作せず、p63にあるappendを使ったmemberで代用しました。

In [40]:
%solve(true).
solve(true) :- !.
solve((A, B)) :- solve(A), solve(B).
solve(A) :- clause(A, B), solve(B).
% memberだとうまく動かなかった。
member2(X, Ys) :- append(As, [X|Xs], Ys).

Previously defined clauses of user:solve/1 were retracted:
:- dynamic solve/1.

solve(true) :-
    !.
solve((A, B)) :-
    solve(A),
    solve(B).
solve(A) :-
    clause(A, B),
    solve(B).

% Asserting clauses for user:solve/1


Previously defined clauses of user:member2/2 were retracted:
:- dynamic member2/2.

member2(A, B) :-
    append(_, [A|_], B).

% Asserting clauses for user:member2/2


Pure Prologの動かしてみます。[a, b, c]のメンバーが順番に出力されます。

In [41]:
?- solve(member2(X, [a, b, c])).
?- retry.
?- retry.

X = a

% Retrying goal: solve(member2(X,[a,b,c]))


X = b

% Retrying goal: solve(member2(X,[a,b,c]))


X = c

### Pure Prologの証明木の構築
Pure Prologの実行時の証明木を構築するを試してみます。Pure Prologと同様にsolve(true, true).にはカットオペレータを追加しました。

In [ ]:
solve(true, true) :- !.
solve((A, B), (ProofA, ProofB)) :- solve(A, ProofA), solve(B, ProofB).
solve(A, (A :- ProofB)) :- clause(A, B), solve(B, ProofB).

息子(son)を確認する例で試してみます。

In [ ]:
son(X, Y) :- father(Y, X), male(X).

そのまま実行するとtrueとのみ出力されます。

In [ ]:
?- son(lot, haran).

証明木構築版のsolveを実行すると、son(lot, haran)の実行過程の証明木の構築過程が出力され、分かりやすいです。

In [ ]:
?- solve(son(lot, haran), Proof).

### 完全版Prologメタ・インタプリタ
簡易版のPrologメタインタプリタに、否定、２階プログラミング用の述語、カットオペレータ、およびシステム述語を追加した完全版Prologメタ・インタプリタにアップグレードします。

理由はわからないですが、前出のsolveとコンフリクトしているのかエラーとなってしまうため、solve_proof.ipynbにて実行しました。

### トレーサ
本物のtraceと名前が衝突するので、以下ではtrace2として実装します。

In [ ]:
trace2(Goal) :- trace2(Goal, 0).
%trace2(true, Depth).
trace2(true, Depth) :- !.
trace2((A, B), Depth) :- trace2(A, Depth), trace2(B, Depth).
trace2(A, Depth) :-
    clause(A, B),
    display2(A, Depth),
    Depth1 is Depth + 1,
    trace2(B, Depth1).
display2(A, Depth) :-
    tab(Depth), write(A), nl.


In [ ]:
?- trace2(member2(X, [a, b, c])).
?- retry.
?- retry.

### エキスパート・システム用改良メタ・インタプリタ
例）オーブン内の配置に関するエキスパートシステム

pastryは、「焼き物」の意味です。

In [33]:
place_in_oven(Dish, top) :-
    pastry(Dish), size(Dish, small).
place_in_oven(Dish, middle) :-
    pastry(Dish), size(Dish, big).
place_in_oven(Dish, middle) :-
    main_male(Dish).
place_in_oven(Dish, low) :-
    slow_cooker(Dish).
pastry(Dish) :- type(Dish, cake).
pastry(Dish) :- type(Dish, bread).
main_meal(Dish) :- type(Dish, meat).
slow_cooker(Dish) :- type(Dish, milk_pudding).

% Asserting clauses for user:place_in_oven/2


% Asserting clauses for user:pastry/1


% Asserting clauses for user:main_meal/1


% Asserting clauses for user:slow_cooker/1


In [35]:
known(A) :- A.

% Asserting clauses for user:known/1


In [37]:
?- set_prolog_flag(unknown, fail).
?- known(type(dish1, cake)).

true

false

## デバッグ
19.3節のメタ・インタプリタを使ったデバッグ手法は、とても有益です。
- 停止しないバグ
- 嘘（偽）の解を返すバグ
- 解を返すことに失敗したバグ


### 無限ループ
無限ループ（スタックオーバフロー）でエラーになる場合の対処方法としてメタ・インタプリタ（solve）にオーバフローのチェックを入れる例が紹介されている。

prologのスタックオーバフローの原因は、再帰処理が延々と続くときであり、clauseで分解されたHeadとTailの分解回数Dを元にオーバフローを検出しするものです。この時、補助情報として処理の途中経過をoverflows内の配列に保持します。
```prolog
solve(A) :- clause(A, B), solve(B).
```
これを以下の様に変更します。
```prolog
solve(A, D, Overflow) :-
    D > 0,
    clause(A, B),
    D is D - 1,
    solve(B, D1, OverflowB),
    return_overflow(OverflowB, A, Overflow).
```
return_overflowでSystem(X)以外の処理をoverflows(S)のSに追加していきます。

In [ ]:
system(A := B).         system(A < B).
system(A > B).          system(A >= B).
system(read(X)).        system(write(X)).
system(integer(X)).     system(functor(T, F, N)).
system(clause(A, B)).   system(system(X)).
system(A is B).         system(writeln(X)).
system(not(A)).  

In [ ]:
solveD(true, D, no_overflow) :- !.
solveD(A, 0, overflow([])).
solveD((A, B), D, Overflow) :-
    D > 0,
    solveD(A, D, OverflowA),
    solve_conjunction(OverflowA, B, D, Overflow).
solveD(A, D, Overflow) :-
    D > 0,
    % clause(2 < 2, B)でエラーになるため、clause(A, B)の前にnot(system(A))を挿入
    not(system(A)),
    clause(A, B),
    D1 is D - 1,
    solveD(B, D1, OverflowB),
    return_overflow(OverflowB, A, Overflow).
solveD(A, D, no_overflow) :-
    D > 0,
    system(A), A.
solve_conjunction(overflow(S), B, D, overflow(S)).
solve_conjunction(no_overflow, B, D, Overflow) :-
    solveD(B, D, Overflow).
return_overflow(no_overflow, A, no_overflow).
return_overflow(overflow(S), A, overflow([A|S])).

### 停止しない挿入isort
確認のために停止しないisortを定義します。


In [ ]:
% 停止しないソート
isort([X|Xs], Ys) :- isort(Xs, Zs), insert(X, Zs, Ys).
isort([], []).
insert(X, [Y|Ys], [X, Y|Ys]) :-
    X < Y.
insert(X, [Y|Ys], [Y|Zs]) :-
    X  >= Y, insert(Y, [X|Ys], Zs).
insert(X, [], [X]).

無限ループするisortを実行してみます。

In [ ]:
%?- isort([2, 2], Xs).

In [ ]:
?- solveD(isort([2, 2], Xs), 6, Overflow).

Overflowのinsert(2, [2], ...)同じ入力(2, [2])で繰り返し呼び出されています。

```prolog
Xs = [2,2,2,2,2,2],
Overflow = overflow([
    isort([2,2],[2,2,2,2,2,2]),
    insert(2,[2],[2,2,2,2,2,2]),
    insert(2,[2],[2,2,2,2,2]),
    insert(2,[2],[2,2,2,2]),
    insert(2,[2],[2,2,2]),
    insert(2,[2],[2,2])])
```

プログラム19.12のままｍだとclause(2 < 2, B)でエラーになるため、clause(A, B)の前にnot(system(A))を挿入しました。

In [ ]:
%?- clause(2 < 2, B).

### 偽な解への対応


In [ ]:
solve(true, true).
solve((A, B), (ProofA, ProofB)) :- 
    solve(A, ProofA), 
    solve(B, ProofB).
solve(A, (A :- ProofB)) :- 
    not(system(A)),
    clause(A, B), 
    solve(B, ProofB).
solve(A, (A :- true)) :- system(A), !, A.

In [ ]:
false_solution(A, Clause) :- 
    solve(A, Proof),
    false_clause(Proof, Clause).

false_clause(true, ok).
false_clause((A, B), Clause) :-
    false_clause(A, ClauseA),
    check_conjunction(ClauseA, B, Clause).
false_clause((A :- B), Clause) :-
    false_clause(B, ClauseB),
    check_clause(ClauseB, A, B, Clause).

check_conjunction(ok, B, Clause) :-
    false_clause(B, Clause).
check_conjunction((A :- B1), B, (A :- B1)).

check_clause(ok, A, B, Clause) :-
    query_goal(A, Answer),
    check_answer(Answer, A, B, Clause).
check_clause((A1 :- B1), A, B, (A1 :- B1)).

check_answer(true, A, B, ok).
check_answer(false, A, B, (A :- B1)) :- 
    extract_body(B, B1).

extract_body(true, true).
extract_body((A :- B), A).
extract_body(((A :- B), Bs), (A, As)) :-
    extract_body(Bs, As).

query_goal(A, true) :- system(A).
query_goal(Goal, Answer) :-
    not(system(Goal)),
    writeln(['Is the Goal', Goal, 'true?']),
    read(Answer).

In [ ]:
% 不完全な挿入ソート
isort([X|Xs], Ys) :- isort(Xs, Zs), insert(X, Zs, Ys).
isort([], []).
insert(X, [Y|Ys], [X, Y|Ys]) :-
    X >= Y.
insert(X, [Y|Ys], [Y|Zs]) :-
    X  > Y, insert(X, Ys, Zs).
insert(X, [], [X]).

In [ ]:
?- isort([3, 2, 1], X).

In [ ]:
?- solveD(false_solution(isort([3, 2, 1], X), C), 10, Overflow).

```prolog
Overflow = overflow([
    false_solution(isort([3,2,1],X),C),solve(isort([3,2,1],X),(isort([3,2,1],X):-
        (isort([2,1],[2,1]):-
            (isort([1],[1]):-
                (isort([],[]):-true),
                (insert(1,[],[1]):-true)),
                (insert(2,[1],[2,1]):-
                    (2>=1:-_41162))),_40516)),
                    solve((isort([2,1],[2,1]),
                    insert(3,[2,1],X)),
                    ((isort([2,1],[2,1]):-
                        (isort([1],[1]):-
                            (isort([],[]):-true),
                            (insert(1,[],[1]):-true)),
                            (insert(2,[1],[2,1]):-
                                (2>=1:-_41162))),_40516)),
                                solve(isort([2,1],[2,1]),
                                (isort([2,1],[2,1]):-
                                    (isort([1],[1]):-
                                        (isort([],[]):-true),
                                        (insert(1,[],[1]):-true)),
                                        (insert(2,[1],[2,1]):-
                                            (2>=1:-_41162)))),
                                            solve((isort([1],[1]),
                                            insert(2,[1],[2,1])),
                                            ((isort([1],[1]):-
                                                (isort([],[]):-true),(insert(1,[],[1]):-true)),(insert(2,[1],[2,1]):-(2>=1:-_41162)))),solve(insert(2,[1],[2,1]),(insert(2,[1],[2,1]):-(2>=1:-_41162))),solve(2>=1,(2>=1:-_41162)),(not(system(2>=1)),clause(2>=1,_41176),solve(_41176,_41162)),call((not(system(2>=1)),clause(2>=1,_41176),solve(_41176,_41162))),call((not(system(2>=1)),clause(2>=1,_41176),solve(_41176,_41162)))])
```

readがnotebookでは動かないみたい！

In [76]:
%?- false_solution(isort([3, 2, 1], X), C).

false_solution.plをターミナルウィンドウで実行します。
insertの３番目の配列が昇順でない場合に、false.を入力すると、障害を発生した箇所でのXと節Cが表示されます。

```bash
% swipl false_solution.pl
?- false_solution(isort([3, 2, 1], X), C).
[Is the Goal,isort([],[]),true?]
|: true.
[Is the Goal,insert(1,[],[1]),true?]
|: true.
[Is the Goal,isort([1],[1]),true?]
|: true.
[Is the Goal,insert(2,[1],[2,1]),true?]
|: false.

X = [3, 2, 1],
C =  (insert(2, [1], [2, 1]):-2>=1) .

?- ctrl-D
```

## 23章　コンパイラ

In [1]:
compile(Tokens, ObjectCode) :-
    parse(Tokens, Structure),
    encode(Structure, Dictionary, Code),
    assemble(Code, Dictionary, ObjectCode).

% Asserting clauses for user:compile/2


### パーザの作成
パーザは、プログラムを解析し、構文木を作成します。

In [2]:
parse(Source, Structure) :-
    pl_program(Structure, Source, []).
pl_program(S) --> [program], identifier(X), [';'], statement(S).

% Asserting clauses for user:parse/2


% Asserting clauses for user:pl_program/3


```prolog
pl_program(Structure, Source, [])
```
と呼ばれているのは、"-->" による限定節文法で本文の差分リストSource\\[]が以下の様にprologに変換されているためです。
```prolog
pl_program(Structure, Source, [])
```
これは、pl_programのリストを表示することで確認できます。

In [3]:
?- listing(pl_program).

:- dynamic pl_program/3.

pl_program(C, [program|A], E) :-
    identifier(_, A, B),
    B=[;|D],
    statement(C, D, E).


true

In [4]:
statement((S ; Ss)) -->
    [begin], statement(S), rest_statements(Ss).
statement(assign(X, V)) -->
    identifier(X), [':='], expression(V).
statement(if(T, S1, S2)) -->
    [if], test(T), [then], statement(S1), [else], statement(S2).
statement(while(T, S)) -->
    [while], test(T), [do], statement(S).
statement(read(X)) -->
    [read], identifier(X).
statement(write(X)) -->
    [write], expression(X).

rest_statements((S ; Ss)) --> 
    [';'], statement(S), rest_statements(Ss).
rest_statements(void) --> [end].

expression(X) --> pl_constant(X).
expression(expr(Op, X, Y)) -->
    pl_constant(X), arithmetic_op(Op), expression(Y).

arithmetic_op('+') --> ['+'].
arithmetic_op('-') --> ['-'].
arithmetic_op('*') --> ['*'].
arithmetic_op('/') --> ['/'].

pl_constant(name(X)) --> identifier(X).
pl_constant(number(X)) --> pl_integer(X).

identifier(X) --> [X], { atom(X) }.
pl_integer(X) --> [X], { integer(X) }.

test(compare(Op, X, Y)) -->
    expression(X), comparision_op(Op), expression(Y).

comparision_op('=') --> ['='].
comparision_op('<>') --> ['<>'].
comparision_op('>') --> ['>'].
comparision_op('<') --> ['<'].
comparision_op('>=') --> ['>='].
comparision_op('<=') --> ['<='].

% Asserting clauses for user:statement/3


% Asserting clauses for user:rest_statements/3


% Asserting clauses for user:expression/3


% Asserting clauses for user:arithmetic_op/3


% Asserting clauses for user:pl_constant/3


% Asserting clauses for user:identifier/3


% Asserting clauses for user:pl_integer/3


% Asserting clauses for user:test/3


% Asserting clauses for user:comparision_op/3


### エンコーダの作成
エンコーダは、構文木からアセンブリ言語を出力します。

In [5]:
encode((X;Xs), D, (Y;Ys)) :-
    encode(X, D, Y), encode(Xs, D, Ys).
encode(void, D, no_op).
encode(assign(Name, E), D, (Code; instr(store, Address))) :-
    lookup(Name, D, Address), encode_expression(E, D, Code).
encode(if(Test, Then, Else), D, (TestCode; ThenCode; instr(jump, L2); label(L1); ElseCode; label(L2))) :-
    encode_test(Test, L1, D, TestCode),
    encode(Then, D, ThenCode),
    encode(Else, D, ElseCode).
encode(while(Test, Do), D, (label(L1); TestCode; DoCode; instr(jump, L1); label(L2))) :-
    encode_test(Test, L2, D, TestCode),
    encode(Do, D, DoCode).
encode(read(X), D, instr(read, Address)) :-
    lookup(X, D, Address).
encode(write(E), D, (Code; instr(write, 0))) :-
    encode_expression(E, D, Code).

% Asserting clauses for user:encode/3


In [6]:
encode_expression(number(C), D, instr(loadc, C)).
encode_expression(name(X), D, instr(load, Address)) :-
    lookup(X, D, Address).
encode_expression(expr(Op, E1, E2), D, (Load; Instruction)) :-
    single_instruction(Op, E2, D, Instruction),
    encode_expression(E1, D, Load).
encode_expression(expr(Op, E1, E2), D, Code) :-
    not(single_instruction(Op, E2, D, Instruction)),
    single_operation(Op, E1, D, E2Code, Code),
    encode_expression(E2, D, E2Code).
single_instruction(Op, number(C), D, instr(OpCode, C)) :-
    literal_operation(Op, OpCode), lookup(X, D, A).
single_instruction(Op, name(X), D, instr(OpCode, A)) :-
    memory_operation(Op, OpCode),
    lookup(X, D, A).
single_operation(Op, E, D, Code, (Code; Instruction)) :-
    commutative(Op),
    single_instruction(Op, E, D, Instruction).
single_operation(Op, E, D, Code, (Code; instr(store, Address); Load; instr(OpCode, Address))) :-
    not(commutative(Op)),
    lookup('temp', D, Address),
    encode_expression(E, D, Load),
    op_code(OP, E, OpCode).
op_code(Op, number(C), OpCode) :- 
    literal_operation(Oop, OpCode).
op_code(Op, name(X), OpCode) :-
    memory_operation(Op, OpCode).

% Asserting clauses for user:encode_expression/3


% Asserting clauses for user:single_instruction/4


% Asserting clauses for user:single_operation/5


% Asserting clauses for user:op_code/3


In [7]:
literal_operation('+', addc).       memory_operation('+', add).
literal_operation('-', subc).       memory_operation('-', sub).
literal_operation('*', mulc).       memory_operation('*', mul).
literal_operation('.', divc).       memory_operation('/', div).
commutative('+').                   commutative('*').

encode_test(compare(Op, E1, E2), Label, D, (Code; instr(OpCode, Label))) :-
    comparison_opcode(Op, OpCode),
    encode_expression(expr('-', E1, E2), D, Code).
% 逆のオペコードを出力している
comparison_opcode('=', jumpne).     comparison_opcode('<>', jumpeq).
comparison_opcode('>', jumple).     comparison_opcode('>=', jumplt).
comparison_opcode('<', jumpge).     comparison_opcode('<=', jumpgt).

%swi-prologではインスタンス化されていない変数との比較は、@<を使用する
lookup(Key, dict(Key, X, Left, Right), Value) :-
    !, X = Value.
lookup(Key, dict(Key1, X, Left, Right), Value) :-
    Key @< Key1, lookup(Key, Left, Value).
lookup(Key, dict(Key1, X, Left, Right), Value) :-
    Key1 @< Key, lookup(Key, Right, Value).


% Asserting clauses for user:literal_operation/2


% Asserting clauses for user:memory_operation/2


% Asserting clauses for user:commutative/1


% Asserting clauses for user:encode_test/4


% Asserting clauses for user:comparison_opcode/2


% Asserting clauses for user:lookup/3


### アセンブラの作成
アセンブラは、アセンブリ言語をマシン命令に変換します。

In [8]:
assemble(Code, Dictionary, TidyCode) :-
    tidy_and_count(Code, 1, N, TidyCode, (instr(halt, 0); block(L))),
    N1 is N + 1,
    allocate(Dictionary, N1, N2),
    L is N2 - N1, !.
tidy_and_count((Code1; Code2), M, N, TCode1, TCode2) :-
    tidy_and_count(Code1, M, M1, TCode1, Rest),
    tidy_and_count(Code2, M1, N, Rest, TCode2).
tidy_and_count(instr(X, Y), N, N1, (instr(X, Y); Code), Code) :-
    N1 is N + 1.
tidy_and_count(label(N), N, N, Code, Code).
tidy_and_count(no_op, N, N, Code, Code).
allocate(void, N, N).
allocate(dict(Name, N1, Before, After), N0, N) :-
    allocate(Before, N0, N1),
    N2 is N1 + 1,
    allocate(After, N2, N).

% Asserting clauses for user:assemble/3


% Asserting clauses for user:tidy_and_count/5


% Asserting clauses for user:allocate/3


### 動作確認
最初はもっとも単純な文法で動きを確認します。

```Pascal
program test0;
begin
    x := 1
end
```

In [9]:
?- parse([program, test0, ';', begin, x, ':=', 1, end], S).

S = assign(x,number(1));void

parseで失敗する場合は、末端の非終端記号pl_constantから遡って、動作を確認するとエラー箇所が掴みやすいです。

In [10]:
?- pl_constant(S, [1], []).

S = number(1)

次に、エンコーダ、アセンブラに繋いでみます。

In [11]:
?- parse([program, test0, ';', begin, x, ':=', 1, end], S), encode(S, Dict, RCode), assemble(RCode, Dict, OpCode).

S = assign(x,number(1));void,
Dict = dict(x,4,void,void),
RCode = (instr(loadc,1);instr(store,4));no_op,
OpCode = instr(loadc,1);instr(store,4);instr(halt,0);block(1)

### swi-prolog固有の問題
途中、lookupのatomの比較で期待通りに動作しなく、その原因がインスタンス化されていない変数との比較によるものと
判明し、lookup定義を修正しました。

```prolog
lookup(Key, dict(Key, X, Left, Right), Value) :-
    !, X = Value.
lookup(Key, dict(Key1, X, Left, Right), Value) :-
    Key @< Key1, lookup(Key, Left, Value).
lookup(Key, dict(Key1, X, Left, Right), Value) :-
    Key1 @< Key, lookup(Key, Right, Value).
```

In [12]:
?- lookup(x, D, V1), lookup(y, D, V2), lookup(x, D, V3), lookup(y, D, V4).

D = dict(x,V1,_28934,dict(y,V2,_28944,_28946)),
V3 = V1,
V4 = V2

### テキストの例題の動作確認
次に、23章で紹介されている例題をコンパイルしてみます。

最初にtest1をコンパイルします。

In [13]:
?- parse([program, test1, ';', begin, write, x, '+', y, '-', z, '/', 2, end], S), encode(S, Dict, RCode), assemble(RCode, Dict, OpCode).

S = write(expr(+,name(x),expr(-,name(y),expr(/,name(z),number(2)))));void,
Dict = dict(x,12,dict(temp,11,void,void),dict(y,13,void,dict(z,14,void,void))),
RCode = ((((instr(loadc,2);instr(store,11);instr(load,14);instr(add,11));instr(store,11);instr(load,13);instr(add,11));instr(add,12));instr(write,0));no_op,
OpCode = instr(loadc,2);instr(store,11);instr(load,14);instr(add,11);instr(store,11);instr(load,13);instr(add,11);instr(add,12);instr(write,0);instr(halt,0);block(4)

次に、test2をコンパイルします。

In [14]:
?- parse([program, test2, ';', begin, if, a, '>', b, then, max, ':=', a, else, max, ':=', b, end], S), encode(S, Dict, RCode), assemble(RCode, Dict, OpCode).

S = if(compare(>,name(a),name(b)),assign(max,name(a)),assign(max,name(b)));void,
Dict = dict(b,11,dict(a,10,void,void),dict(max,12,void,void)),
RCode = (((instr(load,10);instr(sub,11));instr(jumple,7));(instr(load,10);instr(store,12));instr(jump,9);label(7);(instr(load,11);instr(store,12));label(9));no_op,
OpCode = instr(load,10);instr(sub,11);instr(jumple,7);instr(load,10);instr(store,12);instr(jump,9);instr(load,11);instr(store,12);instr(halt,0);block(3)

最後に、test3をコンパイルします。

In [15]:
?- parse(
    [program, factorial, ';',
        begin,
            read, value, ';',
            count, ':=', 1, ';',
            result, ':=', 1, ';',
            while, count, '<', value, do,
                begin,
                    count, ':=', count, '+', 1, ';',
                    result, ':=', result, '*', count,
                end, ';',
            write, result,
        end], S), encode(S, Dict, RCode), assemble(RCode, Dict, OpCode).

S = read(value);assign(count,number(1));assign(result,number(1));while(compare(<,name(count),name(value)),(assign(count,expr(+,name(count),number(1)));assign(result,expr(*,name(result),name(count)));void));write(name(result));void,
Dict = dict(value,21,dict(count,19,void,dict(result,20,void,void)),void),
RCode = instr(read,21);(instr(loadc,1);instr(store,19));(instr(loadc,1);instr(store,20));(label(6);((instr(load,19);instr(sub,21));instr(jumpge,16));(((instr(load,19);instr(addc,1));instr(store,19));((instr(load,20);instr(mul,19));instr(store,20));no_op);instr(jump,6);label(16));(instr(load,20);instr(write,0));no_op,
OpCode = instr(read,21);instr(loadc,1);instr(store,19);instr(loadc,1);instr(store,20);instr(load,19);instr(sub,21);instr(jumpge,16);instr(load,19);instr(addc,1);instr(store,19);instr(load,20);instr(mul,19);instr(store,20);instr(jump,6);instr(load,20);instr(write,0);instr(halt,0);block(3)

構文解析の結果を字下げをつけて読みやすくすると、
```prolog
read(value);assign(count,number(1));assign(result,number(1));
    while(compare(<,name(count),name(value)),
        (assign(count,expr(+,name(count),number(1)));
        assign(result,expr(*,name(result),name(count)));void));
        write(name(result));void
```

出力されたコードを読みやすくすると、以下の様になります。この出力が図23.3と対応しています。
```text
instr(read,21);
instr(loadc,1);
instr(store,19);
instr(loadc,1);
instr(store,20);
instr(load,19);
instr(sub,21);
instr(jumpge,16);
instr(load,19);
instr(addc,1);
instr(store,19);
instr(load,20);
instr(mul,19);
instr(store,20);
instr(jump,6);
instr(load,20);
instr(write,0);
instr(halt,0);
block(3)
```